In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

In [2]:
x = [[2,]]
m = tf.matmul(x, x)
print(m)
print(m.numpy())

tf.Tensor([[4]], shape=(1, 1), dtype=int32)
[[4]]


In [3]:
a = tf.constant([[1,2],
                 [3,4]])
print(a)
print(a.numpy())

tf.Tensor(
[[1 2]
 [3 4]], shape=(2, 2), dtype=int32)
[[1 2]
 [3 4]]


In [4]:
b = tf.add(a, 1)
print(b)

tf.Tensor(
[[2 3]
 [4 5]], shape=(2, 2), dtype=int32)


In [5]:
d = np.array([[5,6],
             [7,8]])
print(a+b)
print((a+b).numpy())

tf.Tensor(
[[3 5]
 [7 9]], shape=(2, 2), dtype=int32)
[[3 5]
 [7 9]]


In [6]:
c = tf.multiply(a, b)
print(c)

tf.Tensor(
[[ 2  6]
 [12 20]], shape=(2, 2), dtype=int32)


In [7]:
num = tf.convert_to_tensor(10)
print(num)

tf.Tensor(10, shape=(), dtype=int32)


In [8]:
for i in range(num.numpy()):
    i = tf.constant(i)
    if int(i % 2) == 0:
        print('even')
    else:
        print('odd')

v = tf.Variable(0.0)
print(v+1)
print((v+1).numpy())

even
odd
even
odd
even
odd
even
odd
even
odd
tf.Tensor(1.0, shape=(), dtype=float32)
1.0


In [9]:
v.assign(5)
print(v)

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>


In [10]:
v.assign_add(1)
print(v)

print(v.read_value())

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=6.0>
tf.Tensor(6.0, shape=(), dtype=float32)


In [11]:
w = tf.Variable([[1.0]])
with tf.GradientTape() as t:
    loss = w * w

grad = t.gradient(loss, w)
print(grad)

tf.Tensor([[2.]], shape=(1, 1), dtype=float32)


In [12]:
w = tf.constant(3.0)
with tf.GradientTape() as t:
    t.watch(w)
    loss = w * w

grad = t.gradient(loss, w)
print(grad)

tf.Tensor(6.0, shape=(), dtype=float32)


In [13]:
w = tf.constant(3.0)
with tf.GradientTape(persistent=True) as t:
    t.watch(w)
    y = w * w
    z = y * y
dy_dw = t.gradient(y, w)
print(dy_dw)

tf.Tensor(6.0, shape=(), dtype=float32)


In [14]:
dz_dw = t.gradient(z, w)
print(dz_dw)

tf.Tensor(108.0, shape=(), dtype=float32)


In [15]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
print(train_images.shape)
train_images = tf.expand_dims(train_images, -1)
test_images = tf.expand_dims(test_images, -1)
print(train_images.shape)

(60000, 28, 28)
(60000, 28, 28, 1)


In [16]:
train_images = tf.cast(train_images / 255, tf.float32)
test_images = tf.cast(test_images / 255, tf.float32)

train_labels = tf.cast(train_labels, tf.int64)
test_labels = tf.cast(test_labels, tf.int64)

dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
print(dataset)

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>


In [17]:
dataset = dataset.shuffle(10000).batch(32)
test_dataset = test_dataset.batch(32)
print(dataset)

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>


In [18]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3, 3], activation='relu', input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(32, [3,3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10)
])

In [19]:
optimizer = tf.keras.optimizers.Adam()
loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
features, labels = next(iter(dataset))
print(features.shape)
print(labels.shape)

(32, 28, 28, 1)
(32,)


In [20]:
predictions = model(features)
print(predictions.shape)

(32, 10)


In [21]:
print(tf.argmax(predictions, axis=1))
print(labels)

tf.Tensor([5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5 5], shape=(32,), dtype=int64)
tf.Tensor([4 9 0 7 0 4 0 5 3 7 2 6 9 9 4 5 8 7 7 8 2 3 0 0 1 8 6 8 7 5 0 0], shape=(32,), dtype=int64)


In [22]:
def loss(model, x, y):
    y_ = model(x)
    return loss_func(y, y_)

train_loss = tf.keras.metrics.Mean('train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('train_accuracy')

test_loss = tf.keras.metrics.Mean('test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy('test_accuracy')

In [23]:
def train_step(model, images, labels):
    with tf.GradientTape() as t:
        pred = model(images)
        loss_step = loss_func(labels, pred)
        # loss_step = loss(model, images, labels)
    grads = t.gradient(loss_step, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    train_loss(loss_step)
    train_accuracy(labels, pred)

In [24]:
def test_step(model, images, labels):
    pred = model(images)
    loss_step = loss_func(labels, pred)
    test_loss(loss_step)
    test_accuracy(labels, pred)

In [25]:
def train():
    for epoch in range(1):
        for (batch, (images, labels)) in enumerate(dataset):
            train_step(model, images, labels)
        print('Epoch {} loss is {}, accuracy is {}'.format(epoch,train_loss.result(), train_accuracy.result()))

        for (batch, (images, labels)) in enumerate(test_dataset):
            test_step(model, images, labels)
        print('Epoch {} loss is {}, accuracy is {}'.format(epoch, test_loss.result(), test_accuracy.result()))

        train_loss.reset_states()
        train_accuracy.reset_states()

        test_loss.reset_states()
        test_accuracy.reset_states()

train()

Epoch 0 loss is 0.8108764290809631, accuracy is 0.7491499781608582
Epoch 0 loss is 0.3923456072807312, accuracy is 0.8737000226974487


In [26]:
m = tf.keras.metrics.Mean('acc')
print(m(10))
print(m(20))

tf.Tensor(10.0, shape=(), dtype=float32)
tf.Tensor(15.0, shape=(), dtype=float32)


In [27]:
print(m.result().numpy())
print(m([30, 40]))
print(m.result())

15.0
tf.Tensor(25.0, shape=(), dtype=float32)
tf.Tensor(25.0, shape=(), dtype=float32)


In [28]:
m.reset_states()
print(m(1))
print(m(2))

tf.Tensor(1.0, shape=(), dtype=float32)
tf.Tensor(1.5, shape=(), dtype=float32)


In [29]:
print(m.result().numpy())

1.5


In [30]:
a = tf.keras.metrics.SparseCategoricalAccuracy('acc')
print(a(labels, model(features)))

tf.Tensor(0.9375, shape=(), dtype=float32)


In [31]:
import tensorflow as tf
import datetime
import os

In [32]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
print(train_images.shape)

(60000, 28, 28)


In [33]:
train_images = tf.expand_dims(train_images, -1)
test_images = tf.expand_dims(test_images, -1)
print(train_images.shape)

(60000, 28, 28, 1)


In [34]:
train_images = tf.cast(train_images / 255, tf.float32)
test_images = tf.cast(test_images / 255, tf.float32)

train_labels = tf.cast(train_labels, tf.int64)
test_labels = tf.cast(test_labels, tf.int64)

dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
print(dataset)

<TensorSliceDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>


In [35]:
dataset = dataset.repeat().shuffle(60000).batch(128)
test_dataset = test_dataset.repeat().batch(128)
print(dataset)

<BatchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>


In [36]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16, [3,3], activation='relu', input_shape=(None, None, 1)),
    tf.keras.layers.Conv2D(32, [3,3], activation='relu'),
    tf.keras.layers.GlobalMaxPooling2D(),
    tf.keras.layers.Dense(10, activation='softmax')
])

In [37]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [38]:
log_dir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir, histogram_freq=1)

file_writer = tf.summary.create_file_writer(log_dir + '/lr')
file_writer.set_as_default()

In [39]:
def lr_sche(epoch):
    learning_rate = 0.02
    if epoch > 5:
        learning_rate = 0.02
    if epoch > 10:
        learning_rate = 0.01
    if epoch > 20:
        learning_rate = 0.005
    tf.summary.scalar('learning_rete', data=learning_rate, step=epoch)
    return learning_rate

In [40]:
lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_sche)

In [41]:
model.fit(dataset,
          epochs=7,
          steps_per_epoch=60000//128,
          validation_data=test_dataset,
          validation_steps=10000//128,
          callbacks=[tensorboard_callback, lr_callback]
          )

Epoch 1/7
468/468 [==============================] - 3s 5ms/step - loss: 0.6147 - accuracy: 0.7909 - val_loss: 0.2798 - val_accuracy: 0.9117
Epoch 2/7
468/468 [==============================] - 2s 5ms/step - loss: 0.2897 - accuracy: 0.9072 - val_loss: 0.2714 - val_accuracy: 0.9146
Epoch 3/7
468/468 [==============================] - 2s 5ms/step - loss: 0.2586 - accuracy: 0.9176 - val_loss: 0.2397 - val_accuracy: 0.9255
Epoch 4/7
468/468 [==============================] - 2s 5ms/step - loss: 0.2483 - accuracy: 0.9222 - val_loss: 0.2550 - val_accuracy: 0.9224
Epoch 5/7
468/468 [==============================] - 2s 5ms/step - loss: 0.2349 - accuracy: 0.9259 - val_loss: 0.2191 - val_accuracy: 0.9317
Epoch 6/7
468/468 [==============================] - 2s 5ms/step - loss: 0.2388 - accuracy: 0.9243 - val_loss: 0.2081 - val_accuracy: 0.9326
Epoch 7/7
468/468 [==============================] - 2s 5ms/step - loss: 0.2264 - accuracy: 0.9288 - val_loss: 0.2194 - val_accuracy: 0.9311


In [42]:
# %load_ext tensorboard
# %matplotlib inlinee
# %tensorboard --logdir logs